In [1]:
# Import the necessary modules
from pyspark.sql.functions import *
from ultis import *
from datetime import datetime
from decouple import config

spark = create_spark()

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a95c203a-52d0-4b52-918f-4dcec4841d74;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
:: resolution report :: resolve 545ms :: artifacts dl 25ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-core_2.12;2.4.0 from central in

In [2]:
spark.sql('''
SELECT
    USER_ID,
    FROM_UTC_TIMESTAMP(MIN(CASE_CREATED_AT), 'GMT+7') AS CREATED_AT,
    FROM_UTC_TIMESTAMP(MAX(CASE_CREATED_AT), 'GMT+7') AS LAST_UPDATED_AT,
    FROM_UTC_TIMESTAMP(NOW(), 'GMT+7') AS LAST_EXECUTED_AT
FROM delta.`s3a://bronze/USER_RATING`
GROUP BY USER_ID
''').write.format('delta').mode('overwrite').save('s3a://silver/USER_DIM')

23/10/05 15:01:34 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/10/05 15:02:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
spark.stop()